In [1]:
RKP = "DL031"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set_theme(style="darkgrid")

In [54]:
# Load datasets and rename columns, load all aqi data but specify metro data name
def loadcsv(city="./data/rkpuram.csv"):
    met = pd.read_csv(city,delimiter=';',skiprows=24)
    aqi = pd.read_csv('./data/station_hour.csv')
    print(aqi.columns)
    met.rename(columns={'# Date': 'Date',}, inplace=True)
    met.rename(columns={'UT time': 'Time',}, inplace=True)
    aqi['Time'] = aqi['Datetime'].str[-8:-3]
    aqi['Date'] = aqi['Datetime'].str[0:10]
    stations = ["DL"+str(x).zfill(3) for x in range(1,39)]
    split_aqi = {}
    for i in range(len(stations)):
        split_aqi[stations[i]] = (aqi[aqi['StationId'] == stations[i]])
    return met,aqi,split_aqi
met,aqi,split_aqi = loadcsv()

c:\users\confusement\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Index(['StationId', 'Datetime', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')


In [58]:
aqi.sort_values(by=['PM2.5'])[:20]

,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Time,Date
1416915,KA003,2017-08-03 09:00:00,0.01,NaN,7.83,18.88,NaN,NaN,NaN,5.87,21.47,0.16,0.06,NaN,31.0,Good,09:00,2017-08-03
1565731,KA009,2016-05-20 08:00:00,0.01,NaN,2.38,10.19,7.24,7.62,NaN,4.70,NaN,NaN,NaN,NaN,61.0,Satisfactory,08:00,2016-05-20
1433448,KA003,2019-06-23 06:00:00,0.01,NaN,3.27,16.07,19.31,NaN,0.81,3.77,4.92,1.80,3.54,NaN,109.0,Moderate,06:00,2019-06-23
2347312,TN004,2019-04-15 18:00:00,0.01,NaN,2.06,7.08,9.13,NaN,0.78,6.61,17.84,0.00,9.81,NaN,57.0,Satisfactory,18:00,2019-04-15
2251510,TN001,2020-05-22 00:00:00,0.01,NaN,3.11,0.37,3.02,NaN,0.32,0.04,15.10,0.00,0.00,NaN,152.0,Moderate,00:00,2020-05-22
2251509,TN001,2020-05-21 23:00:00,0.01,NaN,1.42,0.66,1.79,NaN,0.29,0.39,10.59,0.00,0.00,NaN,152.0,Moderate,23:00,2020-05-21
2251508,TN001,2020-05-21 22:00:00,0.01,NaN,0.98,2.33,3.16,NaN,0.32,1.18,101.53,0.00,0.00,NaN,152.0,Moderate,22:00,2020-05-21
2251507,TN001,2020-05-21 21:00:00,0.01,NaN,3.87,1.17,4.71,NaN,0.66,NaN,120.91,0.00,0.00,NaN,152.0,Moderate,21:00,2020-05-21
2251506,TN001,2020-05-21 20:00:00,0.01,NaN,3.95,2.36,6.16,NaN,0.59,0.28,65.33,0.00,0.00,NaN,152.0,Moderate,20:00,2020-05-21
1738368,MH005,2019-08-19 02:00:00,0.01,25.36,12.28,6.74,19.02,NaN,2.34,0.02,6.94,0.23,NaN,NaN,104.0,Moderate,02:00,2019-08-19


In [116]:
# Pre - processing and loading data
class dataset:
    def __init__(self,met,aqi,split_aqi):
            self.metro_data = met
            self.aqi_data = aqi
            self.split_aqi = split_aqi
    def mergedData(self,station,rlist=['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3','AQI'],roll=48,shift=72):
        df_aqi = self.getdf(station)
        df = pd.merge(df_aqi, self.metro_data, how='inner', on=['Date', 'Time'])
        print("Merged Dataset Size",len(df))
        
        #Pre Processing merged Data
        df['Year'] = df['Date'].str[0:4]
        df['Month'] = df['Date'].str[5:7].astype(np.float64)
        df['Day'] = df['Date'].str[8:10].astype(np.float64)
        df['Hour'] = df['Time'].str[0:2]
        
        # TRIG TRANSFORMATIONS
        df['windX'] = np.cos(np.deg2rad(df['Wind direction'])) * df['Wind speed']
        df['windY'] = np.sin(np.deg2rad(df['Wind direction'])) * df['Wind speed']
        df['hourX'] = np.cos((df['Hour'].astype(np.float64)-1)*np.pi/24)
        df['hourY'] = np.sin((df['Hour'].astype(np.float64)-1)*np.pi/24)
        df['MonthX'] = np.cos((df['Month'].astype(np.float64)-1)*np.pi/12)
        df['MonthY'] = np.sin((df['Month'].astype(np.float64)-1)*np.pi/12)
        
        import datetime
        df['Date'] = pd.to_datetime(df['Date'])
        df['isWeekend'] =  (df['Date'].dt.dayofweek>=5).astype(int)
        
        df.interpolate(method='linear', limit=5,inplace=True)
        
        # Drop Additional columns
        df.drop('Benzene', axis=1, inplace=True)
        df.drop('Toluene',axis=1, inplace=True)
        df.drop('Xylene', axis=1,inplace=True)
        df.drop('AQI_Bucket',axis=1,inplace=True)
        df.drop('Datetime',axis=1,inplace=True)
        df.drop('StationId',axis=1,inplace=True)
        df.drop('Short-wave irradiation',axis=1,inplace=True)
        df.drop('Date',axis=1,inplace=True)
        df.drop('Time',axis=1,inplace=True)
        
        # Rolling and shifting 
        print("Size before roll",len(df))
        for i in rlist:
            df[i+'_lag1'] = df[i].shift(24)
            df[i+'_lag2'] = df[i].shift(48)
            df[i+'_lag3'] = df[i].shift(72)
        for i in rlist:
            df[i+"_pred1"] = df[i].shift(-24)
            df[i+"_pred2"] = df[i].shift(-48)
            df[i+"_pred3"] = df[i].shift(-72)
        newlist = rlist + ['Temperature','Relative Humidity','windX','windY']
        for i in newlist:
            for j in range(24):
                df[i+"_t-"+str(j)] = df[i].shift(j)
                df[i+"_t+"+str(j)] = df[i].shift(-j-shift)
        futurelist = ['Year','MonthX','MonthY','hourX','hourY','isWeekend']
        for i in futurelist:
            for j in range(24):
                df[i+"_t-"+str(j)] = df[i].shift(-(shift+23-j))
        df.dropna(inplace=True)
        print("Size after roll",len(df))
        
        return df.copy()
    def getdf(self,station):
        return self.split_aqi[station]
    def plot(self,station):
        df = self.getdf(station)
    def stats(self):
        pass
dat = dataset(met,aqi,split_aqi)
df = dat.mergedData('DL031')
df

Merged Dataset Size 44035
Size before roll 44035
Size after roll 11596


,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,AQI,...,isWeekend_t-14,isWeekend_t-15,isWeekend_t-16,isWeekend_t-17,isWeekend_t-18,isWeekend_t-19,isWeekend_t-20,isWeekend_t-21,isWeekend_t-22,isWeekend_t-23
237,46.00,83.67,6.73,26.92,34.27,19.20,1.63,6.06,34.75,104.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
238,17.50,88.17,7.70,89.27,92.99,24.48,2.43,15.61,38.33,105.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
239,56.33,148.83,8.48,121.65,123.92,28.67,3.17,20.73,46.50,115.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
309,75.50,244.00,9.85,128.17,131.82,16.22,2.43,8.59,41.25,133.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
310,101.50,298.00,7.20,123.02,123.37,13.86,1.84,4.87,45.00,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43935,14.00,51.00,2.42,10.52,7.60,9.17,0.65,9.62,10.07,77.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43936,19.00,67.00,2.32,26.48,15.98,8.03,0.70,16.80,10.25,78.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43937,14.00,79.00,2.33,16.52,10.70,9.88,0.65,19.98,10.88,79.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43938,18.00,84.00,2.80,17.30,11.47,27.35,0.78,23.70,11.80,80.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
# CNN Model Testing as well
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
TIME_SERIES_LENGTH = 24
features = []
rlist=['PM2.5','PM10','NO','NO2','CO','AQI']
newlist = rlist + ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
# newlist = rlist
print(len(newlist))
for j in range(24):
    for i in newlist:
        features.append(i+'_t-'+str(j))
predVector = []
for j in range(24):
    predVector.append('PM2.5_t+'+str(j))
X = df[features]
y = df[predVector]
# print(X)
X.shape
X = np.array(X).reshape(X.shape[0],TIME_SERIES_LENGTH,len(newlist))
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
Xtrain = scaler.fit_transform(Xtrain.reshape(Xtrain.shape[0],TIME_SERIES_LENGTH*len(newlist)))
Xtrain = Xtrain.reshape(Xtrain.shape[0],TIME_SERIES_LENGTH,len(newlist))
Xtest = scaler.transform(Xtest.reshape(Xtest.shape[0],TIME_SERIES_LENGTH*len(newlist)))
Xtest = Xtest.reshape(Xtest.shape[0],TIME_SERIES_LENGTH,len(newlist))
print(Xtrain.shape,Xtest.shape)

16
(7769, 24, 16) (3827, 24, 16)


In [129]:
for it in rlist:
    print(it,np.mean(df[it]),np.std(df[it]))

PM2.5 102.14626081956295 82.29759489131531
PM10 216.3002180308481 141.8134423731434
NO 43.17497931401046 81.89773244893385
NO2 58.702180613527936 41.01196908713821
CO 1.5506782982223413 2.153596798375222
AQI 224.7247650051742 111.16981620228673


In [136]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
model = Sequential()
# model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(TIME_SERIES_LENGTH,len(newlist))))
model.add(Conv1D(64, 3,padding='same',activation='relu',input_shape=(TIME_SERIES_LENGTH,len(newlist))))
model.add(BatchNormalization())
model.add(Conv1D(64, 6,padding='same',activation='relu',input_shape=(TIME_SERIES_LENGTH,len(newlist))))
model.add(BatchNormalization())
model.add(Conv1D(64, 12,padding='same',activation='relu',input_shape=(TIME_SERIES_LENGTH,len(newlist))))
model.add(BatchNormalization())
model.add(Conv1D(64, 12,padding='same',activation='relu',input_shape=(TIME_SERIES_LENGTH,len(newlist))))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
# model.add(LSTM(200,activation='relu',input_shape=(TIME_SERIES_LENGTH,len(newlist))))
# model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(TIME_SERIES_LENGTH,len(newlist))))
# model.add(Dense(200, activation='relu'))
# model.add(Dense(200, activation='relu'))
model.add(Dense(24, activation='relu'))
model.summary()
#Fit
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(Xtrain, ytrain, epochs=200, batch_size=256,  verbose=1, validation_split=0.2)
#Print Accuracy
testPred = model.predict(Xtest)
trainPred = model.predict(Xtrain)
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))
print(mean_absolute_error(testPred, ytest))
print(mean_absolute_error(trainPred, ytrain))

Model: "sequential_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_74 (Conv1D)           (None, 24, 64)            3136      
_________________________________________________________________
batch_normalization_59 (Batc (None, 24, 64)            256       
_________________________________________________________________
conv1d_75 (Conv1D)           (None, 24, 64)            24640     
_________________________________________________________________
batch_normalization_60 (Batc (None, 24, 64)            256       
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 24, 64)            49216     
_________________________________________________________________
batch_normalization_61 (Batc (None, 24, 64)            256       
_________________________________________________________________
conv1d_77 (Conv1D)           (None, 24, 64)          

25/25 [==============================] - 4s 142ms/step - loss: 929.4924 - mse: 929.4924 - mae: 21.3899 - val_loss: 1091.2892 - val_mse: 1091.2892 - val_mae: 23.1267
Epoch 37/200
25/25 [==============================] - 4s 144ms/step - loss: 869.8724 - mse: 869.8724 - mae: 20.6463 - val_loss: 1104.2091 - val_mse: 1104.2091 - val_mae: 23.1079
Epoch 38/200
25/25 [==============================] - 4s 142ms/step - loss: 880.8713 - mse: 880.8713 - mae: 20.8296 - val_loss: 953.8993 - val_mse: 953.8993 - val_mae: 21.6549
Epoch 39/200
25/25 [==============================] - 3s 133ms/step - loss: 882.2114 - mse: 882.2114 - mae: 20.7388 - val_loss: 917.3022 - val_mse: 917.3022 - val_mae: 21.2451
Epoch 40/200
25/25 [==============================] - 3s 122ms/step - loss: 843.0417 - mse: 843.0417 - mae: 20.2308 - val_loss: 910.5870 - val_mse: 910.5870 - val_mae: 21.7387
Epoch 41/200
25/25 [==============================] - 4s 141ms/step - loss: 825.5524 - mse: 825.5524 - mae: 20.2265 - val_loss: 8

25/25 [==============================] - 3s 135ms/step - loss: 426.8420 - mse: 426.8420 - mae: 14.4434 - val_loss: 529.9392 - val_mse: 529.9392 - val_mae: 16.2300
Epoch 83/200
25/25 [==============================] - 4s 141ms/step - loss: 420.7601 - mse: 420.7601 - mae: 14.2964 - val_loss: 574.1749 - val_mse: 574.1749 - val_mae: 16.6094
Epoch 84/200
25/25 [==============================] - 4s 140ms/step - loss: 421.2939 - mse: 421.2939 - mae: 14.3359 - val_loss: 527.3913 - val_mse: 527.3913 - val_mae: 15.9095
Epoch 85/200
25/25 [==============================] - 4s 141ms/step - loss: 411.9684 - mse: 411.9684 - mae: 14.1280 - val_loss: 513.3412 - val_mse: 513.3412 - val_mae: 15.6417
Epoch 86/200
25/25 [==============================] - 3s 137ms/step - loss: 400.5520 - mse: 400.5520 - mae: 13.9294 - val_loss: 532.2399 - val_mse: 532.2399 - val_mae: 16.2235
Epoch 87/200
25/25 [==============================] - 3s 132ms/step - loss: 391.3318 - mse: 391.3318 - mae: 13.7791 - val_loss: 506.4

25/25 [==============================] - 3s 128ms/step - loss: 329.6095 - mse: 329.6095 - mae: 12.8970 - val_loss: 530.9353 - val_mse: 530.9353 - val_mae: 16.1545
Epoch 129/200
25/25 [==============================] - 4s 148ms/step - loss: 317.0052 - mse: 317.0052 - mae: 12.6694 - val_loss: 449.7157 - val_mse: 449.7157 - val_mae: 14.7528
Epoch 130/200
25/25 [==============================] - 4s 147ms/step - loss: 316.6136 - mse: 316.6136 - mae: 12.6505 - val_loss: 444.0021 - val_mse: 444.0021 - val_mae: 14.6827
Epoch 131/200
25/25 [==============================] - 4s 147ms/step - loss: 313.5248 - mse: 313.5248 - mae: 12.5687 - val_loss: 456.5722 - val_mse: 456.5722 - val_mae: 14.7256
Epoch 132/200
25/25 [==============================] - 4s 146ms/step - loss: 311.6879 - mse: 311.6879 - mae: 12.5387 - val_loss: 438.5660 - val_mse: 438.5660 - val_mae: 14.5731
Epoch 133/200
25/25 [==============================] - 3s 132ms/step - loss: 303.1424 - mse: 303.1424 - mae: 12.3294 - val_loss: 

25/25 [==============================] - 4s 146ms/step - loss: 254.8450 - mse: 254.8450 - mae: 11.4963 - val_loss: 412.5302 - val_mse: 412.5302 - val_mae: 14.1438
Epoch 175/200
25/25 [==============================] - 4s 146ms/step - loss: 244.6019 - mse: 244.6019 - mae: 11.3015 - val_loss: 440.3430 - val_mse: 440.3430 - val_mae: 14.7463
Epoch 176/200
25/25 [==============================] - 3s 124ms/step - loss: 256.3004 - mse: 256.3004 - mae: 11.6230 - val_loss: 412.1277 - val_mse: 412.1277 - val_mae: 14.1270
Epoch 177/200
25/25 [==============================] - 3s 130ms/step - loss: 225.2126 - mse: 225.2126 - mae: 10.8585 - val_loss: 398.0594 - val_mse: 398.0594 - val_mae: 13.8814
Epoch 178/200
25/25 [==============================] - 4s 141ms/step - loss: 236.5448 - mse: 236.5448 - mae: 11.0839 - val_loss: 415.3586 - val_mse: 415.3586 - val_mae: 14.2454
Epoch 179/200
25/25 [==============================] - 4s 141ms/step - loss: 233.0331 - mse: 233.0331 - mae: 11.0389 - val_loss: 

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(np.sqrt(history.history['loss']))
plt.plot(np.sqrt(history.history['val_loss']))
plt.title('model loss')
plt.ylabel('RMSE loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression , mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
dfTrain = df[:]
features = ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
features.append("PM2.5_lag1")
features.append("PM2.5_lag2")

features.append("PM10_lag1")
features.append("PM10_lag2")

features.append("NO2_lag1")
features.append("NO2_lag2")

features.append("SO2_lag1")
features.append("SO2_lag2")

features.append("NO_lag1")
features.append("NO_lag2")

features.append("NOx_lag1")
features.append("NOx_lag2")

# features.append("CO_lag1")
# features.append("CO_lag2")

# features.append("O3_lag1")
# features.append("O3_lag2")

# features.append("NH3_lag1")
# features.append("NH3_lag2")

# features = []
# rlist=['PM2.5','PM10','NO','NO2','CO']
# newlist = rlist + ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
# for i in newlist:
#     for j in range(24):
#         features.append(i+'_t-'+str(j))
        
X = df[features]
y = df['PM2.5_pred3']
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
scaler.fit(Xtrain)

reg = LinearRegression().fit(scaler.transform(Xtrain), ytrain)

testPred = reg.predict(scaler.transform(Xtest))
trainPred = reg.predict(scaler.transform(Xtrain))
mse = np.mean((testPred - np.array(ytest))*(testPred - np.array(ytest)))
print(reg.score(scaler.transform(Xtest), ytest))
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))
print(mean_absolute_error(testPred, ytest))
print(mean_absolute_error(trainPred, ytrain))

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression , mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
dfTrain = df[:]
features = ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
features.append("PM2.5_lag1")
features.append("PM2.5_lag2")

features.append("PM10_lag1")
features.append("PM10_lag2")

features.append("NO2_lag1")
features.append("NO2_lag2")

features.append("SO2_lag1")
features.append("SO2_lag2")

features.append("NO_lag1")
features.append("NO_lag2")

features.append("NOx_lag1")
features.append("NOx_lag2")

features.append("CO_lag1")
features.append("CO_lag2")

features.append("O3_lag1")
features.append("O3_lag2")

features.append("NH3_lag1")
features.append("NH3_lag2")

features = []
rlist=['PM2.5','PM10','NO','NO2','CO']
newlist = rlist + ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
for i in newlist:
    for j in range(24):
        features.append(i+'_t-'+str(j))
        
X = df[features]
y = df['PM2.5_pred1']
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
scaler.fit(Xtrain)

reg = MLPRegressor(random_state=1, max_iter=100).fit(scaler.transform(Xtrain), ytrain)

testPred = reg.predict(scaler.transform(Xtest))
trainPred = reg.predict(scaler.transform(Xtrain))
mse = np.mean((testPred - np.array(ytest))*(testPred - np.array(ytest)))
print(reg.score(scaler.transform(Xtest), ytest))
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))

In [126]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression , mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
dfTrain = df[:]
features = ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
features.append("PM2.5_lag1")
features.append("PM2.5_lag2")

features.append("PM10_lag1")
features.append("PM10_lag2")

features.append("NO2_lag1")
features.append("NO2_lag2")

features.append("SO2_lag1")
features.append("SO2_lag2")

features.append("NO_lag1")
features.append("NO_lag2")

features.append("NOx_lag1")
features.append("NOx_lag2")

features.append("CO_lag1")
features.append("CO_lag2")

features.append("O3_lag1")
features.append("O3_lag2")

features.append("NH3_lag1")
features.append("NH3_lag2")

features = []
rlist=['PM2.5','PM10','NO','NO2','CO']
newlist = rlist + ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
for i in newlist:
    for j in range(24):
        features.append(i+'_t-'+str(j))

X = df[features]
y = df['PM2.5_pred1']
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
print(Xtrain.shape,ytrain.shape)
# scaler.fit(Xtrain)

reg = SVR(C=3.0, epsilon=0.2).fit(Xtrain, ytrain)

testPred = reg.predict(Xtest)
trainPred = reg.predict(Xtrain)
mse = np.mean((testPred - np.array(ytest))*(testPred - np.array(ytest)))
print(reg.score(Xtest, ytest))
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))
print(mean_absolute_error(testPred, ytest))
print(mean_absolute_error(trainPred, ytrain))

(7769, 360) (7769,)
0.3379685723175532
65.78093011534429
67.0801815314519
43.41983467544137
43.25189223855267


In [ ]:
import tensorflow as tf  
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
features = ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
features = []
rlist=['PM2.5','PM10','NO','NO2','CO']
newlist = rlist + ['Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']
for i in newlist:
    for j in range(24):
        features.append(i+'_t-'+str(j))
predVector = []
for j in range(24):
    predVector.append('PM2.5_t+'+str(j))
X = df[features]
y = df[predVector]
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
scaler.fit(Xtrain)
print(Xtrain.shape)
model = Sequential()
model.add(Dense(200, input_dim=360, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(24, activation='linear'))
model.summary()
#Fit
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(scaler.transform(Xtrain), ytrain, epochs=100, batch_size=50,  verbose=1, validation_split=0.2)
#Print Accuracy
testPred = model.predict(scaler.transform(Xtest))
trainPred = model.predict(scaler.transform(Xtrain))
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))
print(mean_absolute_error(testPred, ytest))
print(mean_absolute_error(trainPred, ytrain))

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(np.sqrt(history.history['loss']))
plt.plot(np.sqrt(history.history['val_loss']))
plt.title('model loss')
plt.ylabel('RMSE loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()